### Here is the simple steps to train model using NanoNets API

Previously in blog, we have an intro to Object Detection and how to make predection on tensorflow pre-trained model.
Tensorflow pre-trained model has been trained on [COCO dataset](http://cocodataset.org/) which has 90 different objects annotated.

Now, what if you want to detect your choice of objects (may or may not be in 90 classes)? So here, we are going to discuss how can we make custom object detection model using simplest way and with less relative data.


#### Step 1: Data Preparation

What we need? Few images (around 100-200) and their object bounding box annotations. There are many ways to store bounding box annotation, but here I am using XML file to store annotation.

Make a work directiory, and open this notebook in same diretory.
keep all images in one floder and annotation(.xml) in other folder (name of annotation file should be same as corresponding name of image)

Also need to have list of all objects (same name given in annotation).

In [16]:
## change the folder path if needed
image_folder = "data/images"
annotation_folder = "data/annotations"
object_list = [''] ## will refer as categories
extentions = ['.png'] ## list of image extension

#### Step 2: Create NanoNets Account

To use NanoNets API to train model, you first need to create account on nanonets and get your api key from there

In [17]:
AUTH_KEY = "" ## put your api key here

In [18]:
### imports

import requests, json, os
import xml.etree.ElementTree as ET

In [19]:
#### CONSTANTS
BASE_URL = "https://app.nanonets.com/api/v2/ObjectDetection/Model/"
MODEL_URL = "https://app.nanonets.com/api/v2/ObjectDetection/Models/"

#### Step 3: Initialize new model

In [20]:
def create_new_model(categories):
    """ function will create a new model architecture for training
    
    Args:
    categories: List of objects you want to predict
    
    return:
    model_id: a unique reference to new created model
    """
    
    
    payload = json.dumps({"categories" : categories})
    headers = {
        'Content-Type': "application/json",
        }

    response = requests.request("POST", BASE_URL, headers=headers, auth=requests.auth.HTTPBasicAuth(AUTH_KEY, ''), data=payload)

    result = json.loads(response.text)
    model_id, model_type, categories = (result["model_id"], result["model_type"], result["categories"])
    return model_id

In [21]:
model_id = create_new_model(object_list)
print model_id

a6b5947a-241a-41f1-af2d-54413722e16a


#### Step 4: upload data
We have created a new model, now we need to uplaod the data to train model.
Here is the code the upload and xml_file data to model

In [22]:
def get_model_info(model_id):
    """function to get information about model at any time
    Args:
    model_id: unique model_id generated at model creation time
    """
    
    response = requests.request('GET', '%s%s'%(BASE_URL, model_id), auth= requests.auth.HTTPBasicAuth(AUTH_KEY, ''))
    print response.text
    result = json.loads(response.text)
    model_id, model_type, categories, state = (result["model_id"], result["model_type"], result["categories"], result["state"])
    return model_id, model_type, categories, state

In [23]:
def get_xml_file_name(image_file):
    return os.path.join(annotation_folder, "%s.xml"%(image_file.rsplit('.', 1)[0]))

def create_xml_to_object(image_file):
    tree = ET.parse(get_xml_file_name(image_file))
    root = tree.getroot()

    object_list = []
    
    ### here if you find any problem here, just check with xml file format and change the code 
    ### to access member an elements 
    
    for member in root.findall('object'):
        label = member[0].text.lower()
        bndbox = {}
        bndbox['xmin'] = int(member[4][0].text)
        bndbox['ymin'] = int(member[4][1].text)
        bndbox['xmax'] = int(member[4][2].text)
        bndbox['ymax'] = int(member[4][3].text)
        object_list.append({'name': label, 'bndbox': bndbox})
    return object_list

def upload_objects_by_file(model_id):
    image_count = 0
    print "uploading images...."
    for f in os.listdir(image_folder):
        if not f.endswith(tuple(extentions)): continue
        filename = os.path.join(image_folder, f)
        file = open(filename, 'rb')

        object_data = json.dumps(create_xml_to_object(f))

        data = {'file' : file,
                'data' :('', '[{"filename":"%s", "object": %s}]'%(f, object_data)),
                'modelId' :('', '%s'%model_id)}
        response = requests.post('%s%s/UploadFile/'%(BASE_URL, model_id), auth=requests.auth.HTTPBasicAuth(AUTH_KEY, ''), files=data)
        image_count += 1
    print "Number of Uploaded Images : ", image_count
    return get_model_info(model_id)

In [24]:
upload_objects_by_file(model_id)

Number of Uploaded Images :  197
{"model_id":"a6b5947a-241a-41f1-af2d-54413722e16a","model_type":"localization","state":2,"status":"Some data uploaded","accuracy":0,"categories":[{"name":"signature","count":267}]}


(u'a6b5947a-241a-41f1-af2d-54413722e16a',
 u'localization',
 [{u'count': 267, u'name': u'signature'}],
 2)

#### Step 5: Train a model

We have uploaded a data to model, now it's time to train a model. Run the code below, sit back and relax for some time. Training is going on.

In [25]:
def train_model(model_id):

    headers = {'authorization': 'Basic %s'%AUTH_KEY}
    querystring = {'modelId': model_id}
    response = requests.request('POST', '%s%s/Train/'%(BASE_URL, model_id), headers=headers, auth=requests.auth.HTTPBasicAuth(AUTH_KEY, ''), params=querystring)
    print "training started .... "
    print json.loads(response.text)

In [26]:
train_model(model_id)

training started .... 
{u'status': u'In queue for training', u'model_id': u'a6b5947a-241a-41f1-af2d-54413722e16a', u'state': 3, u'model_type': u'localization', u'categories': [{u'count': 267, u'name': u'signature'}], u'accuracy': 0}


While training is going on, you can check the state of model by `get_model_info` function. When training will finished, state of model will change to 4.

In [27]:
get_model_info(model_id)

{"model_id":"a6b5947a-241a-41f1-af2d-54413722e16a","model_type":"localization","state":3,"status":"In queue for training","accuracy":0,"categories":[{"name":"signature","count":267}]}


(u'a6b5947a-241a-41f1-af2d-54413722e16a',
 u'localization',
 [{u'count': 267, u'name': u'signature'}],
 3)

#### step 6: It's time to predict

Keep your test image ready.

In [ ]:
test_image_path = ""

In [ ]:
def predict_single_image(model_id, test_image_path):
    if not test_image_path.endswith(tuple(extentions)):
        print "provide image with correct extentions"
        return 0
    data = {'file': open(filepath, 'rb'),
            'modelId': ('', '%s'%model_id)}
    response = requests.post(url, auth=requests.auth.HTTPBasicAuth(AUTH_KEY, ''), files=data)
    result = json.loads(response.text)
    return result

In [ ]:
predict_single_image(model_id, test_image_path)